In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cur

In [2]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
new_model = "llama-2-7b-custom"
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
fp16 = False
bf16 = False
warmup_ratio = 0.03
group_by_length = True
max_seq_length = None
packing = False
device_map = {"": 0}

In [3]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [15]:
dataset_name = "professional_psychology"

In [16]:
from datasets import load_dataset
ds = load_dataset("cais/mmlu", dataset_name)

test-00000-of-00001.parquet:   0%|          | 0.00/133k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/22.1k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/4.69k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/612 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/69 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [17]:
def get_choices():
    return ["A", "B", "C", "D"]


def format_subject(subject):
    return subject.replace("_", " ")


def format_example(example, include_answer=True):
    # Extract question and choices
    prompt = example['question']
    choices = example['choices']

    # Add choices to the prompt
    for j, choice in enumerate(choices):
        prompt += "\n{}. {}".format(get_choices()[j], choice)

    prompt += "\nAnswer:"

    if include_answer:
        # Get the answer (convert to letter if it's an index)
        answer = example['answer']
        if isinstance(answer, (int, np.integer)):
            answer = get_choices()[answer]
        prompt += " {}\n\n".format(answer)

    return prompt

In [18]:
system_prompt = f"You are an expert in {format_subject(dataset_name)}. You will be presented with a question and four possible answers. Choose the correct answer. Do not output any explanation, only choose the correct option."

In [19]:
def gen_prompt(train_dataset, num_examples=5):
    prompt = f"The following are example multiple choice questions (with answers).\n\n"

    for i in range(min(num_examples, len(train_dataset))):
        prompt += format_example(train_dataset[i])

    return prompt

In [20]:
import re

def extract_answer(generated_text):
    """
    Extract the first letter that matches A, B, C, or D
    """
    # Convert to uppercase to catch both lower and upper case
    generated_text = generated_text.upper()

    # Look for first occurrence of A, B, C, or D
    match = re.search(r'[A-D]', generated_text)

    return match.group(0) if match else None

In [24]:
num_few_shot = 0
total_questions = 100

In [27]:
import numpy as np
from tqdm import tqdm

In [28]:
correct_predictions = 0

few_shot_prompt = gen_prompt(ds['dev'], num_few_shot)
combined_system_prompt = f"{system_prompt}\n\n{few_shot_prompt}"
correct_answers = []
predicted_answers = []

import time

t1 = time.time()

for idx in tqdm(range(total_questions)):
    # Current test question
    test_question = format_example(ds['test'][idx], include_answer=False)

    prompt = f"""[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n {test_question} [/INST]"""

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024)

    result = pipe(prompt)

    text = result[0]['generated_text']

    content = text.split("[/INST]")[1].strip()

    predicted_answer = extract_answer(content)

    ground_truth = ds['test'][idx]['answer']

    # Normalize ground truth
    if isinstance(ground_truth, (int, np.integer)):
        ground_truth = get_choices()[ground_truth]

    predicted_answers.append(predicted_answer)
    correct_answers.append(ground_truth)

    # Compare
    if predicted_answer == ground_truth:
        correct_predictions += 1
    # else:
    #     # Optional: print misclassified examples for debugging
    #     print(f"Predicted: {predicted_answer}, Correct: {ground_truth}")

t2 = time.time()

accuracy = correct_predictions / total_questions
print(f"Accuracy: {accuracy}")

print(f"Time taken: {t2-t1}")

100%|██████████| 100/100 [09:26<00:00,  5.67s/it]

Accuracy: 0.42
Time taken: 566.965891122818


In [29]:
import pandas as pd
df = pd.DataFrame({
    "Question": [format_example(ds['test'][i], include_answer=False) for i in range(total_questions)],
    "Predicted Answer": predicted_answers,
    "Correct Answer": correct_answers
})

df.to_csv(f"{dataset_name}_llama_vanilla_results.csv", index=False)